In [1]:
import pandas as pd
import datetime as dt
import numpy as np
from constants import holding_period
import os

In [2]:
class SBFTradingRule():
    # strategy: hold weight w_big in mkt_rf when dy > dy_cutoff,
    # weight w_small in mkt_rf when dy < dy_cutoff

    w = 0.0625
    
    # Assume 100% of portfolio liquidated each month and repurchased with new quantities
    
    # strategy-specifc columns for trades_df (variables we want to keep track of for subsequent analysis)
    # usually the variables that led the strategy to open trade in the first place
    # In this case, just size
    strategy_specific_trades_df_columns = {'size': pd.Series([], dtype='float')} 
    
    def __init__(self,portfolio_db):
        self.portfolio_db = portfolio_db
    
    # Regardless of the strategy you are implementing, this method must return
    # open_trades_df, close_trades_df
    #
    # open_trades_df is a DataFrame with all the required trades_df columns plus any custom ones for this strategy
    #     each row is a new trade the strategy wants to open
    #     method only populates the security_id (index) and quantity required columns, plus any custom columns. Rest remain NaN to be populated elsewhere
    # close_trades_df is a DataFrame that is a subset of the rows of portfolio_db.trades_df
    #     each row is an exist trade the strategy wants to close
    #     we don't need to populate any columns in this function
    def compute_trades(self,signal_df, date):                
        # Since we are not doing 100% turnover each period, not all currently open trades should be closed
        # Close trades where enough time has passed
        close_trades_df = self.portfolio_db.trades_df[(self.portfolio_db.trades_df['close_datetime'].isna()) & ((date - self.portfolio_db.trades_df['open_datetime']) >= pd.Timedelta(days=holding_period))].copy()
        open_trades_df = self.empty_trades_df()

        # get current size

        if not signal_df.empty:
            buys_df = signal_df[signal_df['security_id'] == 'BTC-USD']
            sells_df = signal_df[signal_df['security_id'] != 'BTC-USD']

            dollars_per_buy = self.portfolio_db.current_nav()*self.w
            buys_df['quantity'] = dollars_per_buy / signal_df['prc']

            dollars_per_sell = -self.portfolio_db.current_nav()*self.w
            sells_df['quantity'] = dollars_per_sell / signal_df['prc']

            # Add new row to the open_trades_df with quantity and security_id
            open_trades_df = pd.concat([self.empty_trades_df(), buys_df.loc[:,['security_id','quantity']], sells_df.loc[:,['security_id','quantity']] ])
        
        return open_trades_df, close_trades_df
    
    # Returns an empty trades_df
    # Used so we know the right columns to populate when creating a trades_df else
    def empty_trades_df(self):
        return pd.concat([self.portfolio_db.empty_trades_df(), pd.DataFrame(self.strategy_specific_trades_df_columns)], axis=1)
        